In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import glob
import os
import pickle
import matplotlib.pyplot as plt
# Printing All Results
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
############################################################################################################################
##########################################       Loading Data        #######################################################
############################################################################################################################

In [5]:
## Function to read a type of adjency matrixes of Indian villages, and then create a list
def adjToGraphlist(name="name"):
    
    # Location of data
    path = r'C:\Users\Mahdi\COVID19-Simulation\IndianVillagesData\1. Network Data\Adjacency Matrices'
    all_files = sorted(glob.glob(path + "/*"+name+"*.csv"),key=os.path.getmtime)


    li = []
    
    # Reading all adjecny matrixes based on the name
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=None)
        li.append(df)
    
    GraphList=list()
    
    #Converting each adj to a networkx garph object
    for i in range(len(li)):
        GraphList.append(nx.from_pandas_adjacency(li[i]))
        
    return(GraphList)

In [6]:
# Name of the different types of the graphs
GraphNames = ["allVillageRelationships","andRelationships",
             "giveadvice","helpdecision",
             "keroricecome","keroricego",
             "borrowmoney","lendmoney",
             "medic","nonrel",
             "rel","templecompany",
             "visitcome","visitgo"]

In [7]:
%%time
## Location to save graphs
path=r"C:\Users\Mahdi\COVID19-Simulation\CreatingGraphs-IndianVillages\Graphs/"

## Creating a graphlist for each graph name and then save that
for name in GraphNames:
    print(name)
    x = adjToGraphlist(name)
    with open(path+name+"_Graphs", 'wb') as fp:
        pickle.dump(x, fp) 

allVillageRelationships
andRelationships
giveadvice
helpdecision
keroricecome
keroricego
borrowmoney
lendmoney
medic
nonrel
rel
templecompany
visitcome
visitgo
Wall time: 1min 59s


In [8]:
%%time
## Reading Graphs
path=r'C:\Users\Mahdi\COVID19-Simulation\CreatingGraphs-IndianVillages\Graphs/'

with open(path+"allVillageRelationships_Graphs", 'rb') as fp:
    ALL=pickle.load(fp)
    
with open(path+"andRelationships_Graphs", 'rb') as fp:
    AND=pickle.load(fp)
    
with open(path+"giveadvice_Graphs", 'rb') as fp:
    giveadvice=pickle.load(fp)
    
with open(path+"helpdecision_Graphs", 'rb') as fp:
    helpdecision=pickle.load(fp)
    
with open(path+"keroricecome_Graphs", 'rb') as fp:
    keroricecome=pickle.load(fp)
    
with open(path+"keroricego_Graphs", 'rb') as fp:
    keroricego=pickle.load(fp)
    
with open(path+"borrowmoney_Graphs", 'rb') as fp:
    borrowmoney=pickle.load(fp)
    
with open(path+"lendmoney_Graphs", 'rb') as fp:
    lendmoney=pickle.load(fp)
    
with open(path+"medic_Graphs", 'rb') as fp:
    medic=pickle.load(fp)
    
with open(path+"nonrel_Graphs", 'rb') as fp:
    nonrel=pickle.load(fp)
    
with open(path+"rel_Graphs", 'rb') as fp:
    rel=pickle.load(fp)
    
with open(path+"templecompany_Graphs", 'rb') as fp:
    templecompany=pickle.load(fp)
    
with open(path+"visitcome_Graphs", 'rb') as fp:
    visitcome=pickle.load(fp)
    
with open(path+"visitgo_Graphs", 'rb') as fp:
    visitgo=pickle.load(fp)

Wall time: 1.48 s


In [9]:
%%time
## Composing Graphs for different kind of graphs and the set the weight and lebel attribute

Money_Graph = list()
for i in range(75):
    U = nx.compose(lendmoney[i],borrowmoney[i])
    for (u, v) in U.edges():
        U[u][v]['weight']=1
        U[u][v]['label']="Money"
    Money_Graph.append(U)
    
    
    
Visit_Graph = list()
for i in range(75):
    U = nx.compose(visitcome[i],visitgo[i])
    for (u, v) in U.edges():
        U[u][v]['weight']=1
        U[u][v]['label']="Visit"
    Visit_Graph.append(U)
    
    
    
Kerosene_Graph = list()
for i in range(75):
    U = nx.compose(keroricecome[i],keroricego[i])
    for (u, v) in U.edges():
        U[u][v]['weight']=1
        U[u][v]['label']="Kerosene"
    Kerosene_Graph.append(U)
    
    
    
Temple_Graph = list()
for i in range(75):
    U = templecompany[i]
    for (u, v) in U.edges():
        U[u][v]['weight']=1
        U[u][v]['label']="Temple"
    Temple_Graph.append(U)

Wall time: 2.32 s


In [10]:
%%time
## Creating Multigraph
GraphList = list()
for i in range(75):
    G = nx.MultiGraph()
    G.add_edges_from(Money_Graph[i].edges(data=True))
    G.add_edges_from(Visit_Graph[i].edges(data=True))
    G.add_edges_from(Kerosene_Graph[i].edges(data=True))
    G.add_edges_from(Temple_Graph[i].edges(data=True))
    G.add_nodes_from(Money_Graph[i].nodes())
    
    # For normalizing centrality attribute
    lng = len(G.nodes()) - 1
    
    # Normalizing centrality attribute (from fraction to integer)
    Money_DegCen = nx.degree_centrality(Money_Graph[i])
    Money_DegCen.update((x , int(y*lng))for x, y in Money_DegCen.items())
    # Setting attributes for each node
    nx.set_node_attributes(G, Money_DegCen, 'Money_DegCen')
    
    Visit_DegCen = nx.degree_centrality(Visit_Graph[i])
    Visit_DegCen.update((x , int(y*lng))for x, y in Visit_DegCen.items())
    nx.set_node_attributes(G, Visit_DegCen, 'Visit_DegCen')
    
    Kerosene_DegCen = nx.degree_centrality(Kerosene_Graph[i])
    Kerosene_DegCen.update((x , int(y*lng))for x, y in Kerosene_DegCen.items())
    nx.set_node_attributes(G, Kerosene_DegCen, 'Kerosene_DegCen')
    
    Temple_DegCen = nx.degree_centrality(Temple_Graph[i])
    Temple_DegCen.update((x , int(y*lng))for x, y in Temple_DegCen.items())
    nx.set_node_attributes(G, Temple_DegCen, 'Temple_DegCen')
    
    GraphList.append(G)

# Saving Multigraph
path=r'C:\Users\Mahdi\COVID19-Simulation\CreatingGraphs-IndianVillages\Results/'
with open(path+"GraphList_Multi", 'wb') as fp:
    pickle.dump(GraphList, fp) 

Wall time: 2.64 s


In [9]:
## Creating the transimmsion graph based on the multigraph to run the simulatin on
def TransmissionGraph(Data=GraphList,Labels=["Money","Visit","Kerosene","Temple"],P=[0.2,0.3,0.1,0.25],
                 dropEdge="",dropNode="",dropTop=0):
        
        # Initilizing output 
        out = list()
        # For every Graph in GraphList
        for G in Data:
            if len(dropNode)!=0:
                ############# Filtering Nodes #############
                # A dictionary of the node and the specific characteristics by which we want to filter
                listNodeAttr = nx.get_node_attributes(G,name=dropNode)
                # Find the threshold for that characteristics. Edges from a node with larger char, will drop.
                threshold = np.percentile(list(listNodeAttr.values()),100-dropTop)
                # List of nodes that must be quarantined!
                DropList = [k for k,v in listNodeAttr.items() if v>threshold ]
                # Removing edges from that node. Quarantining!
                for node in DropList:
                    edgelist = list(G.edges(node))
                    G.remove_edges_from(edgelist)

            
            ########### Filtering Edges #############
            if len(dropEdge)!=0:
                # Iterating over all edges
                for u,v,key,atrr in list(G.edges(data=True,keys=True)):
                    # If the edge has a attribute which is in dropEdge list:
                    if (atrr['label'] in  dropEdge) :
                        # We remove that edge
                        G.remove_edge(u,v,key=key)

  
            ############# Correcting the weights of Edges #############
            for u,v,key,atrr in list(G.edges(data=True,keys=True)):
                # Finding location of leble in list --> finding p
                weight = P[Labels.index(atrr['label'])]
                # changing the weight
                G[u][v][key]['weight'] = weight
                
     
            ############# Calculating overall transmission probability #############
            # Iterating over each two nodes that there is at least 1 edge between
            for (u,v) in list(set(G.edges())):
                # Keys : number of edges between this two nodes
                keys = list(G.get_edge_data(u,v).keys())
                # 1 - Transimision probability : each element for one edge
                noTransProb = [(1-G[u][v][i]['weight']) for i in keys]
                # The probability that at least one of the edges infects other people
                newWeight = np.round(1-np.prod(noTransProb),3)
                # Removind old edges: key by key
                for i in keys:
                    G.remove_edge(u,v,key=i)
                # Add new edge, which weight shows the transmission probability
                G.add_edges_from([(u,v)],weight=newWeight)
                
                
                
            # Appending modified Graph    
            out.append(nx.Graph(G))
        
        # Returning output
        return(out)

In [23]:
%%time
# Copying mulitigraph (Otherwise it's chnaged in the each run)
data = [x.copy() for x in GraphList]

# Creating the graph with given P and filteration policy
temp=TransmissionGraph(Data=data,dropNode="Visit_DegCen",dropTop=10,dropEdge="",
                                        Labels=["Money","Visit","Kerosene","Temple"],P=[0.2,0.3,0.1,0.25])

# Saving the graph to run simulatin on
path=r'C:\Users\Mahdi\COVID19-Simulation\File 1 - First Try - Mar27\Results/'
with open(path+"GraphList_DegCen10_V", 'wb') as fp:
    pickle.dump(temp, fp) 

Wall time: 12 s


In [21]:
## Checking the output
i=54
GraphList[i].number_of_nodes()
temp[i].number_of_nodes()


len(set(GraphList[i].edges()))
len(GraphList[i].edges())
temp[i].number_of_edges()


u=list(temp[i].edges())[6][0]
v=list(temp[i].edges())[6][1]
temp[i].get_edge_data(u,v)
GraphList[i].get_edge_data(u,v)

208

208

709

1273

530

{'weight': 0.1}

{0: {'weight': 1, 'label': 'Visit'}, 1: {'weight': 1, 'label': 'Kerosene'}}

In [47]:
####################################################################################################################
####################################################################################################################
####################################################################################################################
####################################################################################################################